In [ ]:
%pip install SpeechRecognition 
%pip install pydub
%pip install openai-whisper
%pip install ffmpeg

In [ ]:
%pip install "numpy<2"

In [ ]:
#conda install -c conda-forge ffmpeg
#need to run this for ffmpeg to work

In [ ]:
import speech_recognition as sr


from pydub import AudioSegment
from pydub.utils import which

AudioSegment.converter = which("ffmpeg")
AudioSegment.ffprobe = which("ffprobe")


# Initialize the recognizer
recognizer = sr.Recognizer()

# Specify the path to your MP3 file
mp3_file = "/Users/rgopalam/Desktop/office/audio2.mp3"

# Convert MP3 to WAV
from pydub import AudioSegment
audio = AudioSegment.from_mp3(mp3_file)
wav_file = "temp_audio.wav"
audio.export(wav_file, format="wav")

# Recognize the speech in the audio file
import whisper

def transcribe_audio(audio_file):
    model = whisper.load_model("base")
    result = model.transcribe(audio_file)
    return result["text"]

transcript = transcribe_audio(wav_file)
print(transcript)



In [1]:
import os
from pydub import AudioSegment
from pydub.utils import which
from google.cloud import speech

# Ensure ffmpeg and ffprobe paths are set
AudioSegment.converter = which("ffmpeg")
AudioSegment.ffprobe = which("ffprobe")

# Set the path to your Google Cloud JSON credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/rgopalam/Desktop/seismic-rarity-427422-p7-ab3b4a8726ef.json"

def get_sample_rate(audio_file):
    """Get the sample rate of the audio file."""
    try:
        audio = AudioSegment.from_file(audio_file)
        return audio.frame_rate
    except Exception as e:
        print(f"Error getting sample rate: {e}")
        return None

def convert_mp3_to_wav(mp3_file, output_file="temp_audio.wav"):
    """Convert an MP3 file to WAV format with a single channel (mono)."""
    try:
        audio = AudioSegment.from_mp3(mp3_file)
        audio = audio.set_channels(1)  # Convert to mono
        audio.export(output_file, format="wav")
        return output_file
    except Exception as e:
        print(f"Error converting MP3 to WAV: {e}")
        return None

def transcribe_audio_google(audio_file):
    """Transcribe audio using Google Cloud Speech-to-Text."""
    try:
        # Initialize the Speech-to-Text client
        client = speech.SpeechClient()

        # Read the audio file
        with open(audio_file, "rb") as audio:
            audio_content = audio.read()

        # Get the sample rate of the WAV file
        sample_rate = get_sample_rate(audio_file)
        if not sample_rate:
            print("Could not determine sample rate.")
            return None

        # Configure the recognition request
        audio_config = speech.RecognitionConfig(
            encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
            sample_rate_hertz=sample_rate,  # Dynamically match the WAV file's sample rate
            language_code="en-US"
        )
        audio_request = speech.RecognitionAudio(content=audio_content)

        # Perform the transcription
        response = client.recognize(config=audio_config, audio=audio_request)

        # Concatenate all transcriptions into one string
        transcript = " ".join([result.alternatives[0].transcript for result in response.results])
        return transcript

    except Exception as e:
        print(f"Error during transcription: {e}")
        return None

# Main execution
if __name__ == "__main__":
    mp3_file = "/Users/rgopalam/Desktop/office/audio2.mp3"
    
    if not os.path.exists(mp3_file):
        print(f"File not found: {mp3_file}")
    else:
        wav_file = convert_mp3_to_wav(mp3_file)
        if wav_file:
            transcript = transcribe_audio_google(wav_file)
            if transcript:
                print("Transcription:")
                print(transcript)
            # Clean up temporary file
            os.remove(wav_file)


Transcription:
from this now I'm loving my shoes these aren't my shoes I'm buying them so you can do whatever you want to them mate go on get in there  get in there you love the fact that this dog is chewing on these  they're tasty they're shoes how much are these shoes 1,800 bucks I'm getting there guys


In [8]:
%pip install ffpyplayer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 7.4 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import yt_dlp
from ffpyplayer.player import MediaPlayer

def play_video_with_audio(video_url):
    # yt-dlp options to fetch the best video URL
    ydl_opts = {
        "format": "best",            # Fetch the best video + audio format
        "quiet": True,               # Suppress output
        "no_warnings": True          # Suppress warnings
    }

    # Fetch video info and get the stream URL
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(video_url, download=False)
        video_stream_url = info["url"]  # Get the direct video URL

    # Initialize OpenCV video capture
    cap = cv2.VideoCapture(video_stream_url)
    if not cap.isOpened():
        print("Error: Cannot open video stream.")
        return

    # Initialize ffpyplayer for audio
    player = MediaPlayer(video_stream_url)

    print("Press 'q' to quit the video stream.")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("End of stream or cannot fetch frame.")
            break

        # Display video frame
        cv2.imshow('YouTube Video Stream', frame)

        # Play audio synchronously
        audio_frame, val = player.get_frame()
        if val != 'eof' and audio_frame:
            img, t = audio_frame

        # Exit on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release resources
    cap.release()
    player.close_player()
    cv2.destroyAllWindows()

# Replace with your YouTube video URL
youtube_url = "https://www.youtube.com/watch?v=96Y6mc3C1Bg"  # Example video
play_video_with_audio(youtube_url)


Press 'q' to quit the video stream.


In [1]:

import os
import subprocess
from google.cloud import speech
import yt_dlp

# Set the path to your Google Cloud JSON credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/rgopalam/Desktop/seismic-rarity-427422-p7-ab3b4a8726ef.json"

def get_youtube_audio_stream(video_url):
    """Fetch the direct audio stream URL using yt-dlp."""
    ydl_opts = {
        "format": "bestaudio/best",  # Fetch the best audio format
        "quiet": True,
        "no_warnings": True,
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(video_url, download=False)
        return info["url"]

def transcribe_stream(audio_url):
    """Stream audio from a YouTube video and transcribe it using Google Cloud Speech-to-Text."""
    # Initialize Google Cloud Speech-to-Text client
    client = speech.SpeechClient()

    # Start ffmpeg to extract the audio stream
    ffmpeg_command = [
        "ffmpeg",
        "-i", audio_url,                # Input: YouTube audio URL
        "-f", "s16le",                  # Output format: signed 16-bit little-endian PCM
        "-ac", "1",                     # Audio channels: mono
        "-ar", "16000",                 # Sample rate: 16 kHz
        "-loglevel", "quiet",           # Suppress ffmpeg logs
        "pipe:1"                        # Output to stdout
    ]

    process = subprocess.Popen(ffmpeg_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, bufsize=10**8)

    # Configure the Google Cloud StreamingRecognizeRequest
    streaming_config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code="en-US",
    )
    streaming_request = speech.StreamingRecognitionConfig(config=streaming_config, interim_results=True)

    # Generator to yield audio chunks
    def audio_generator():
        while True:
            data = process.stdout.read(4096)  # Read audio chunks
            if not data:
                break
            yield data

    # Perform streaming recognition
    requests = (speech.StreamingRecognizeRequest(audio_content=chunk) for chunk in audio_generator())
    responses = client.streaming_recognize(config=streaming_request, requests=requests)

    # Process and display the transcription results
    try:
        for response in responses:
            for result in response.results:
                if result.is_final:
                    print("Transcript:", result.alternatives[0].transcript)
    except Exception as e:
        print(f"Error during transcription: {e}")
    finally:
        process.terminate()

# Main execution
if __name__ == "__main__":
    youtube_url = "https://www.youtube.com/watch?v=96Y6mc3C1Bg"  # Replace with your YouTube video URL
    audio_stream_url = get_youtube_audio_stream(youtube_url)
    if audio_stream_url:
        transcribe_stream(audio_stream_url)


Transcript: you and I love crime junkies love love that's like our go-to podcast when we're doing our road trips
Transcript:  we're obsessed
Transcript:  yes
Transcript:  so true crime is a huge Niche I feel like


KeyboardInterrupt: 

In [4]:
import os
import subprocess
from google.cloud import speech
import yt_dlp

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/rgopalam/Desktop/seismic-rarity-427422-p7-ab3b4a8726ef.json"

def test_audio_transcription(youtube_url):
    ydl_opts = {"format": "bestaudio/best", "quiet": True, "no_warnings": True}
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(youtube_url, download=False)
        audio_url = info["url"]
        print(f"Audio URL: {audio_url}")
    
    ffmpeg_command = [
        "ffmpeg", "-i", audio_url, "-f", "s16le", "-ac", "1", "-ar", "16000",
        "-loglevel", "quiet", "pipe:1"
    ]
    process = subprocess.Popen(ffmpeg_command, stdout=subprocess.PIPE)

    client = speech.SpeechClient()
    streaming_config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code="en-US",
    )
    streaming_request = speech.StreamingRecognitionConfig(config=streaming_config, interim_results=True)

    def audio_generator():
        while True:
            data = process.stdout.read(4096)
            if not data:
                break
            yield data

    requests = (speech.StreamingRecognizeRequest(audio_content=chunk) for chunk in audio_generator())
    responses = client.streaming_recognize(config=streaming_request, requests=requests)

    try:
        for response in responses:
            for result in response.results:
                if result.is_final:
                    print("[Transcript]:", result.alternatives[0].transcript)
    except Exception as e:
        print(f"Transcription error: {e}")
    finally:
        process.terminate()

test_audio_transcription("https://www.youtube.com/watch?v=96Y6mc3C1Bg")


Audio URL: https://rr1---sn-n4v7snls.googlevideo.com/videoplayback?expire=1737550968&ei=GJiQZ5D5BaKYsfIP6aGKwAM&ip=2601%3A644%3A601%3Afa0%3Adc96%3Abed9%3Aae26%3A6f2b&id=o-AI0til_thuib6YUmxPGHMF5yFZbdppdWSXNXKL6AVB02&itag=251&source=youtube&requiressl=yes&xpc=EgVo2aDSNQ%3D%3D&met=1737529368%2C&mh=KN&mm=31%2C29&mn=sn-n4v7snls%2Csn-o097znzr&ms=au%2Crdu&mv=m&mvi=1&pl=48&rms=au%2Cau&initcwndbps=4688750&bui=AY2Et-MIpz93XyyhqfTNZOJQo13TH19SKWPVHvpedTHMf93SfZD28A8eMbM1HxppXRFZLcg-mnhpsY8o&spc=9kzgDT6_Y8JbOhzsAjiOVFExD_Uq1NDLK2vBPGztNupszll4XTm6&vprv=1&svpuc=1&mime=audio%2Fwebm&rqh=1&gir=yes&clen=75946061&dur=5104.141&lmt=1729252675319032&mt=1737529009&fvip=5&keepalive=yes&fexp=51326932%2C51335594%2C51353498%2C51371294%2C51384461&c=IOS&txp=5532434&sparams=expire%2Cei%2Cip%2Cid%2Citag%2Csource%2Crequiressl%2Cxpc%2Cbui%2Cspc%2Cvprv%2Csvpuc%2Cmime%2Crqh%2Cgir%2Cclen%2Cdur%2Clmt&sig=AJfQdSswRQIhAJoyTO9_5IySG065uDhuFsUBUoJIlK9k6AOdsfnVP3xyAiB6Y9zsYaFH6-nUxd1E37FjVrGZuRk0ar34bxkDOBigVg%3D%3D&lsparams

KeyboardInterrupt: 

In [1]:
import os
import subprocess
import cv2
import yt_dlp
from google.cloud import speech

# Set Google Cloud credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/rgopalam/Desktop/seismic-rarity-427422-p7-ab3b4a8726ef.json"

def transcribe_audio_stream(audio_url):
    """Stream audio for transcription using Google Cloud Speech-to-Text."""
    client = speech.SpeechClient()

    # Use ffmpeg to convert audio stream to raw PCM data
    ffmpeg_command = [
        "ffmpeg", "-i", audio_url, "-f", "s16le", "-ac", "1", "-ar", "16000",
        "-loglevel", "quiet", "pipe:1"
    ]
    process = subprocess.Popen(ffmpeg_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    streaming_config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code="en-US",
    )
    streaming_request = speech.StreamingRecognitionConfig(config=streaming_config, interim_results=True)

    def audio_generator():
        while True:
            data = process.stdout.read(4096)
            if not data:
                break
            yield data

    requests = (speech.StreamingRecognizeRequest(audio_content=chunk) for chunk in audio_generator())
    responses = client.streaming_recognize(config=streaming_request, requests=requests)

    try:
        for response in responses:
            for result in response.results:
                if result.is_final:
                    print("[Transcript]:", result.alternatives[0].transcript)
    except Exception as e:
        print(f"Transcription error: {e}")
    finally:
        process.terminate()

def play_video_with_transcription(video_url, audio_url):
    """Play video using OpenCV while transcribing audio."""
    cap = cv2.VideoCapture(video_url)
    if not cap.isOpened():
        print("Error: Could not open video stream.")
        return

    print("Press 'q' to quit the video stream.")

    # Start transcription in a background process
    import threading
    transcription_thread = threading.Thread(target=transcribe_audio_stream, args=(audio_url,))
    transcription_thread.start()

    while True:
        ret, frame = cap.read()
        if not ret:
            print("End of video stream.")
            break

        # Display video
        cv2.imshow("YouTube Video Stream", frame)

        # Exit on 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # Wait for transcription to complete
    transcription_thread.join()

if __name__ == "__main__":
    youtube_url = "https://www.youtube.com/watch?v=96Y6mc3C1Bg"

    # Fetch video and audio URLs
    ydl_opts = {"format": "best", "quiet": True, "no_warnings": True}
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(youtube_url, download=False)
        video_url = info["url"]

    audio_opts = {"format": "bestaudio/best", "quiet": True, "no_warnings": True}
    with yt_dlp.YoutubeDL(audio_opts) as ydl:
        audio_info = ydl.extract_info(youtube_url, download=False)
        audio_url = audio_info["url"]

    # Play video with transcription
    play_video_with_transcription(video_url, audio_url)



Press 'q' to quit the video stream.
[Transcript]: you and I love crime junkies love love that's like our go-to podcast when we're doing our road trips
[Transcript]:  we're obsessed
[Transcript]:  yes
[Transcript]:  so true crime is a huge Niche I feel like
[Transcript]:  everyone listens to it and if you don't like you hear about it I don't know so with it finally being October and things get a little spookier I figured
[Transcript]:  I'm going to test my limits as a podcast host and try to dabble with True Crime let's go I'm terrified oh I'm excited it's a lot of pressure
[Transcript]:  no I can't compare to Ashley know it was just so fun like let's go okay okay well let's do it


KeyboardInterrupt: 

: 

In [ ]:
import os
import subprocess
import cv2
import yt_dlp
from ffpyplayer.player import MediaPlayer
from google.cloud import speech
from queue import Queue
import threading
import time

# Set Google Cloud credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/rgopalam/Desktop/seismic-rarity-427422-p7-ab3b4a8726ef.json"

def transcribe_audio_stream(audio_url, transcription_queue):
    """Stream audio for transcription using Google Cloud Speech-to-Text."""
    client = speech.SpeechClient()

    # Use ffmpeg to convert audio stream to raw PCM data
    ffmpeg_command = [
        "ffmpeg", "-i", audio_url, "-f", "s16le", "-ac", "1", "-ar", "16000",
        "-loglevel", "quiet", "pipe:1"
    ]
    process = subprocess.Popen(ffmpeg_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    streaming_config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code="en-US",
    )
    streaming_request = speech.StreamingRecognitionConfig(config=streaming_config, interim_results=True)

    def audio_generator():
        while True:
            data = process.stdout.read(4096)
            if not data:
                break
            yield data

    requests = (speech.StreamingRecognizeRequest(audio_content=chunk) for chunk in audio_generator())
    responses = client.streaming_recognize(config=streaming_request, requests=requests)

    try:
        for response in responses:
            for result in response.results:
                if result.is_final:
                    transcription = result.alternatives[0].transcript
                    timestamp = time.time()  # Record when the transcription was generated
                    transcription_queue.put((timestamp, transcription))
    except Exception as e:
        print(f"Transcription error: {e}")
    finally:
        process.terminate()

def play_video_with_audio_and_transcription(video_url):
    """Play video with synchronized audio and perform real-time transcription."""
    # yt-dlp options to fetch the best video URL
    ydl_opts = {
        "format": "best",            # Fetch the best video + audio format
        "quiet": True,               # Suppress output
        "no_warnings": True          # Suppress warnings
    }

    # Fetch video info and get the stream URL
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(video_url, download=False)
        video_stream_url = info["url"]

    # Fetch audio URL
    audio_opts = {"format": "bestaudio/best", "quiet": True, "no_warnings": True}
    with yt_dlp.YoutubeDL(audio_opts) as ydl:
        audio_info = ydl.extract_info(video_url, download=False)
        audio_stream_url = audio_info["url"]

    # Initialize OpenCV video capture
    cap = cv2.VideoCapture(video_stream_url)
    if not cap.isOpened():
        print("Error: Cannot open video stream.")
        return

    # Initialize ffpyplayer for audio
    player = MediaPlayer(video_stream_url)

    # Queue for synchronized transcription
    transcription_queue = Queue()

    # Start transcription in a background thread
    transcription_thread = threading.Thread(
        target=transcribe_audio_stream, args=(audio_stream_url, transcription_queue)
    )
    transcription_thread.start()

    print("Press 'q' to quit the video stream.")
    while True:
        ret, frame = cap.read()
        if not ret:
            print("End of stream or cannot fetch frame.")
            break

        # Display video frame
        cv2.imshow('YouTube Video Stream', frame)

        # Play audio synchronously
        audio_frame, val = player.get_frame()
        if val != 'eof' and audio_frame:
            _, timestamp = audio_frame

            # Synchronize and display transcription
            while not transcription_queue.empty():
                transcription_time, transcription = transcription_queue.queue[0]
                if transcription_time <= time.time():  # Check if it's time to display the transcription
                    print("[Transcript]:", transcription)
                    transcription_queue.get()
                else:
                    break

        # Exit on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release resources
    cap.release()
    player.close_player()
    cv2.destroyAllWindows()

    # Wait for transcription thread to complete
    transcription_thread.join()

# Replace with your YouTube video URL
youtube_url = "https://www.youtube.com/watch?v=96Y6mc3C1Bg"  # Example video
play_video_with_audio_and_transcription(youtube_url)


Press 'q' to quit the video stream.
[Transcript]: you and I love crime junkies love love that's like our go-to podcast when we're doing our road trips
[Transcript]:  we're obsessed
[Transcript]:  yes
[Transcript]:  so true crime is a huge Niche I feel like
[Transcript]:  everyone listens to it and if you don't like you hear about it I don't know so with it finally being October and things get a little spookier I figured
[Transcript]:  I'm going to test my limits as a podcast host and try to dabble with True Crime let's go I'm terrified oh I'm excited it's a lot of pressure
[Transcript]:  no I can't compare to Ashley know it was just so fun like let's go okay okay well let's do it
[Transcript]:  hi guys welcome to another episode of 2 hot takes I'm your host Morgan I'm Justin
[Transcript]:  trigger warning today guys first stories coming at you fucking aggressively and Trigger warning mentions of murder Gore rape um it is very graphic
[Transcript]:  and if you even skip that first story

In [1]:
import os
import subprocess
import cv2
import yt_dlp
from ffpyplayer.player import MediaPlayer
from google.cloud import speech
from queue import Queue
import threading
import time

# Set Google Cloud credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/rgopalam/Desktop/seismic-rarity-427422-p7-ab3b4a8726ef.json"

def transcribe_audio_stream(audio_url, transcription_queue):
    """Stream audio for transcription using Google Cloud Speech-to-Text."""
    client = speech.SpeechClient()

    # Use ffmpeg to convert audio stream to raw PCM data
    ffmpeg_command = [
        "ffmpeg", "-i", audio_url, "-f", "s16le", "-ac", "1", "-ar", "16000",
        "-loglevel", "quiet", "pipe:1"
    ]
    process = subprocess.Popen(ffmpeg_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    streaming_config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code="en-US",
    )
    streaming_request = speech.StreamingRecognitionConfig(config=streaming_config, interim_results=True)

    def audio_generator():
        while True:
            data = process.stdout.read(4096)
            if not data:
                break
            yield data

    requests = (speech.StreamingRecognizeRequest(audio_content=chunk) for chunk in audio_generator())
    responses = client.streaming_recognize(config=streaming_request, requests=requests)

    try:
        for response in responses:
            for result in response.results:
                if result.is_final:
                    transcription = result.alternatives[0].transcript
                    timestamp = time.time()  # Record when the transcription was generated
                    transcription_queue.put((timestamp, transcription))
    except Exception as e:
        print(f"Transcription error: {e}")
    finally:
        process.terminate()

def play_video_with_audio_and_transcription(video_url):
    """Play video with synchronized audio and perform real-time transcription."""
    # yt-dlp options to fetch the best video URL
    ydl_opts = {
        "format": "best",            # Fetch the best video + audio format
        "quiet": True,               # Suppress output
        "no_warnings": True          # Suppress warnings
    }

    # Fetch video info and get the stream URL
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(video_url, download=False)
        video_stream_url = info["url"]

    # Fetch audio URL
    audio_opts = {"format": "bestaudio/best", "quiet": True, "no_warnings": True}
    with yt_dlp.YoutubeDL(audio_opts) as ydl:
        audio_info = ydl.extract_info(video_url, download=False)
        audio_stream_url = audio_info["url"]

    # Initialize OpenCV video capture
    cap = cv2.VideoCapture(video_stream_url)
    if not cap.isOpened():
        print("Error: Cannot open video stream.")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)  # Get frame rate
    frame_delay = int(1000 / fps)  # Delay between frames in milliseconds

    # Initialize ffpyplayer for audio
    player = MediaPlayer(video_stream_url)

    # Queue for synchronized transcription
    transcription_queue = Queue()

    # Start transcription in a background thread
    transcription_thread = threading.Thread(
        target=transcribe_audio_stream, args=(audio_stream_url, transcription_queue)
    )
    transcription_thread.start()

    print("Press 'q' to quit the video stream.")
    while True:
        ret, frame = cap.read()
        if not ret:
            print("End of stream or cannot fetch frame.")
            break

        # Display video frame
        cv2.imshow('YouTube Video Stream', frame)

        # Play audio synchronously
        audio_frame, val = player.get_frame()
        if val != 'eof' and audio_frame:
            _, timestamp = audio_frame

            # Synchronize and display transcription
            while not transcription_queue.empty():
                transcription_time, transcription = transcription_queue.queue[0]
                if transcription_time <= time.time():  # Check if it's time to display the transcription
                    print("[Transcript]:", transcription)
                    transcription_queue.get()
                else:
                    break

        # Exit on pressing 'q'
        if cv2.waitKey(frame_delay) & 0xFF == ord('q'):
            break

    # Release resources
    cap.release()
    player.close_player()
    cv2.destroyAllWindows()

    # Wait for transcription thread to complete
    transcription_thread.join()

# Replace with your YouTube video URL
youtube_url = "https://www.youtube.com/watch?v=96Y6mc3C1Bg"  # Example video
play_video_with_audio_and_transcription(youtube_url)


Press 'q' to quit the video stream.
[Transcript]: you and I love crime junkies love love that's like our go-to podcast when we're doing our road trips
[Transcript]:  we're obsessed
[Transcript]:  yes
[Transcript]:  so true crime is a huge Niche I feel like
[Transcript]:  everyone listens to it and if you don't like you hear about it I don't know so with it finally being October and things get a little spookier I figured


KeyboardInterrupt: 

: 